In [13]:
# Copyright 2022 The Forta Foundation
 
import requests
import pandas as pd
import json
import dotenv
import os

# Load environment variables
dotenv.load_dotenv()

True

In [14]:
def execute_query(sql: str) -> pd.DataFrame:
        url = "https://q.luabase.com/run"
        payload = {"block":
                   {"details": {
                    "sql": sql,
                    }
                    },
                   "api_key": os.environ.get('LUABASE_API_KEY')
                   }

        headers = {"content-type": "application/json"}
        response = requests.request("POST", url, json=payload, headers=headers, timeout=600)  # 10min timeout
        
        data = response.json()
        
        print(data)
        return pd.DataFrame(data["data"])

This notebook allows to backtest the attack detector V2 against a set of alerts. Independent of the threshold set within the bot, the log will be analyzed, so the relevant alerts, their anomaly score, overall anomaly score, and any FP filtering mitigations are shown.

Since there is a command line character limit, the local persistence mechanism needs to be enabled to persist state across executions (by setting local node property in constants.py to 1)

In [15]:
start_date = "2022-12-01" #when the attack detector first started to operate

attacker_addresses = "0xf3a465c9fa6663ff50794c698f600faa4b05c777,0x2ffc59d32a524611bb891cab759112a51f9e33c0" #comma separated list of attacker addresses (note, while all alerts from those addresses are pushed to the attack detector, one should check the clustering bot to see whether they are clustered together)
chain = "ethereum" #ethereum or binance 
name = "ANKR"
end_date = "2022-12-04" #should be the date of the attack

# attacker_addresses = "0x001B91c794dFEecf00124D3F9525DD32870B6ee9" #comma separated list of attacker addresses (note, while all alerts from those addresses are pushed to the attack detector, one should check the clustering bot to see whether they are clustered together)
# chain = "ethereum" #ethereum or binance 
# name = "CryptoRubric"
# end_date = "2022-12-28" #should be the date of the attack

# attacker_addresses = "0x3bdF01ed32F07e8e843163b5d478d4502F5743CD,0x25fDe76A52D01c83E31d2d3D5e1d2011ff103c56,0xBeAdeDBABED6A353c9cAa4894Aa7E5F883e32967" #comma separated list of attacker addresses (note, while all alerts from those addresses are pushed to the attack detector, one should check the clustering bot to see whether they are clustered together)
# chain = "ethereum" #ethereum or binance 
# name = "ElasticSwap"
# start_date = "2022-11-30" #when the attack detector first started to operate
# end_date = "2022-12-15" #should be the date of the attack

# attacker_addresses = "0x86Aa1c46f2Ae35ba1B228dC69fB726813D95b597,0x3c4e5b099f3c02122079d124138377e1b9048629" #comma separated list of attacker addresses (note, while all alerts from those addresses are pushed to the attack detector, one should check the clustering bot to see whether they are clustered together)
# chain = "binance" #ethereum or binance 
# name = "Nimbus"
# end_date = "2022-12-15" #should be the date of the attack

# attacker_addresses = "0xF4FD2EbE7196c8E99E88bcc4Aef69dda0e493B8F" #comma separated list of attacker addresses (note, while all alerts from those addresses are pushed to the attack detector, one should check the clustering bot to see whether they are clustered together)
# chain = "binance" #ethereum or binance 
# name = "BlackGold"
# end_date = "2022-12-15" #should be the date of the attack

# attacker_addresses = "0x286E09932B8D096cbA3423d12965042736b8F850" #comma separated list of attacker addresses (note, while all alerts from those addresses are pushed to the attack detector, one should check the clustering bot to see whether they are clustered together)
# chain = "binance" #ethereum or binance 
# name = "aespool"
# end_date = "2022-12-17" #should be the date of the attack

# attacker_addresses = "0xf71708C59be7e32B9FF2aa174F07311869C6bf0c" #comma separated list of attacker addresses (note, while all alerts from those addresses are pushed to the attack detector, one should check the clustering bot to see whether they are clustered together)
# chain = "binance" #ethereum or binance 
# name = "optionroom"
# end_date = "2022-12-17" #should be the date of the attack




In [16]:
#clustering
for attacker_address in attacker_addresses.split(","):
    sql = f"SELECT metadata FROM forta.{chain}_alerts WHERE bot_id = '0xd3061db4662d5b3406b52b20f34234e462d2c275b99414d76dc644e2486be3e9' and CAST(substring(block_timestamp,1,19) as datetime)  >= '2022-10-01T00:00:00' AND CAST(substring(block_timestamp,1,19)  as datetime)  <= '{end_date}T00:00:00' and  lower(arrayStringConcat(metadata)) like '%{attacker_address.lower()}%'"
    print(sql)
    df = execute_query(sql)
    print(df)

SELECT metadata FROM forta.ethereum_alerts WHERE bot_id = '0xd3061db4662d5b3406b52b20f34234e462d2c275b99414d76dc644e2486be3e9' and CAST(substring(block_timestamp,1,19) as datetime)  >= '2022-10-01T00:00:00' AND CAST(substring(block_timestamp,1,19)  as datetime)  <= '2022-12-04T00:00:00' and  lower(arrayStringConcat(metadata)) like '%0xf3a465c9fa6663ff50794c698f600faa4b05c777%'
{'data': [], 'ok': True, 'error': None, 'hit_max_row': False, 'rows': 0, 'meta': [{'name': 'metadata', 'type': 'Array(Map(String, Nullable(String)))'}], 'parameters': {}, 'start_time': '2022-12-30T23:49:15.997267+00:00', 'statistics': {'applied_limit': False, 'blocks': 0, 'bytes': 42565280250, 'calculated_rows_before_limit': 1, 'elapsed': 1.449, 'rows_before_limit': 0, 'rows': 319503139, 'total_rows': 319503139, 'written_bytes': 0, 'written_rows': 0}, 'time_elapsed': 1.479, 'trace': 'projects/luabase/traces/925661ad123b659b1c3ea565c6d95583'}
Empty DataFrame
Columns: []
Index: []
SELECT metadata FROM forta.ethereu

In [17]:
alert_hashes_df = pd.DataFrame(columns=["alert_hash","block_timestamp"])
for attacker_address in attacker_addresses.split(","):
    sql = f"select alert_hash, block_timestamp FROM forta.{chain}_alerts WHERE CAST(substring(block_timestamp,1,19) as datetime)  >= '{start_date}T00:00:00' AND CAST(substring(block_timestamp,1,19)  as datetime)  <= '{end_date}T00:00:00' and lower(arrayStringConcat(addresses)) like '%{attacker_address.lower()}%'"
    print(sql)
    df = execute_query(sql)
    alert_hashes_df = pd.concat([alert_hashes_df, df])

len(alert_hashes_df)

select alert_hash, block_timestamp FROM forta.ethereum_alerts WHERE CAST(substring(block_timestamp,1,19) as datetime)  >= '2022-12-01T00:00:00' AND CAST(substring(block_timestamp,1,19)  as datetime)  <= '2022-12-04T00:00:00' and lower(arrayStringConcat(addresses)) like '%0xf3a465c9fa6663ff50794c698f600faa4b05c777%'
{'data': [{'alert_hash': '0xa49b71c9553a3aba0f51c95807d88296d5e3815efa45d6c0720531570db4a518', 'block_timestamp': '2022-12-02T11:11:47Z'}, {'alert_hash': '0xb77076018fcc1e18bab4df662d180fba1b412ef40081102c4e9953040c6d7342', 'block_timestamp': '2022-12-02T10:12:23Z'}, {'alert_hash': '0xfc70ecd414dbdd720c5e3e2ce95150820db0fd372899f995d6f97918695a5569', 'block_timestamp': '2022-12-02T11:39:23Z'}, {'alert_hash': '0x625d7eeeccf006bd81d70e52025309ff774023a714bbd88d74dae5afcc0e004e', 'block_timestamp': '2022-12-02T01:58:35Z'}, {'alert_hash': '0x04fbd57fc436158a73c8f26dd0ddf3cb07097367866f6b6ecc39394f6d0363de', 'block_timestamp': '2022-12-02T11:50:59Z'}, {'alert_hash': '0xd74240d6a1

1429

In [18]:
alert_hashes_df

,alert_hash,block_timestamp
0,0xa49b71c9553a3aba0f51c95807d88296d5e3815efa45...,2022-12-02T11:11:47Z
1,0xb77076018fcc1e18bab4df662d180fba1b412ef40081...,2022-12-02T10:12:23Z
2,0xfc70ecd414dbdd720c5e3e2ce95150820db0fd372899...,2022-12-02T11:39:23Z
3,0x625d7eeeccf006bd81d70e52025309ff774023a714bb...,2022-12-02T01:58:35Z
4,0x04fbd57fc436158a73c8f26dd0ddf3cb07097367866f...,2022-12-02T11:50:59Z
...,...,...
97,0xe97dea1abbe7945c8bde1a59e0f062d490c839c8027b...,2022-12-02T03:17:23Z
98,0xcac5cd519b5d3c6c30f158759c189e24c2e2476c2508...,2022-12-02T08:16:47Z
99,0xa9f1844e905a752b856259002c8355c9e5ce6b1c3301...,2022-12-02T08:14:11Z
100,0xd46f1386a782472cbbfa050328f984559acb93b3858a...,2022-12-02T03:18:59Z


In [25]:
#remove all _key files (L2 cache)
!echo 'output.txt' > {name}.txt

#run npm run to get and route to output.txt
count = 0
alert_hashes = ""
for alert_hash in alert_hashes_df["alert_hash"]:
    count += 1
    alert_hashes += f",{alert_hash}"
    if count % 90 == 0:
        !npm run alert {alert_hashes} >> {name}.txt 2>> {name}.txt
        alert_hashes = ""
if count % 90 != 0:
    !npm run alert {alert_hashes} >> {name}.txt 2>> {name}.txt



In [20]:
!grep 'ERROR' {name}.txt


ERROR: Error: Request failed with status code 504
ERROR: RangeError: Invalid string length
ERROR: Error: Request failed with status code 504
2022-12-30 16:12:32,070 - root - ERROR - Failed to populate denominator cache for 1 transfer-in 2022-11-30 16:54:23 2022-12-01 16:54:23: HTTPSConnectionPool(host='q.luabase.com', port=443): Read timed out.
ERROR: RangeError: Invalid string length
ERROR: RangeError: Invalid string length
ERROR: RangeError: Invalid string length
ERROR: RangeError: Invalid string length
ERROR: RangeError: Invalid string length
ERROR: Error: Request failed with status code 504
ERROR: RangeError: Invalid string length
ERROR: RangeError: Invalid string length
ERROR: RangeError: Invalid string length
ERROR: RangeError: Invalid string length
ERROR: RangeError: Invalid string length
ERROR: RangeError: Invalid string length
ERROR: RangeError: Invalid string length


In [21]:
!grep 'WARN' {name}.txt

2022-12-30 16:12:44,532 - root - WARNING - Exception in get_denominator Denominator not found for 1 transfer-in 2022-11-30 16:54:23 2022-12-01 16:54:23 in cache
2022-12-30 16:12:44,997 - root - WARNING - Exception in get_denominator Denominator not found for 1 transfer-in 2022-11-30 16:54:23 2022-12-01 16:54:23 in cache


In [22]:
!grep 'adding cluster mapping' {name}.txt

In [23]:
!grep 'adding FP mitigation cluster' {name}.txt

2022-12-30 15:59:03,351 - root - INFO - alert 0xeb5a429c14f5adeeee13cac734dfeab72b6e252983b3f6ba95cc9433e747c47a adding FP mitigation cluster: 0xe421b5c6680bb06f25e0f8dd138c96913d2c1599. FP mitigation clusters size now: 0
2022-12-30 16:51:56,753 - root - INFO - alert 0x6157d00fa9e98278d2af2a8a0d2c4983cc9dc1335f5b12a479149ac2e3265e54 adding FP mitigation cluster: 0x2ffc59d32a524611bb891cab759112a51f9e33c0. FP mitigation clusters size now: 1
2022-12-30 16:53:45,279 - root - INFO - alert 0xf39e06ddaf5b79d44e98e00370e83c14e3a68342e11f1c264e91cc40d9ccce84 adding FP mitigation cluster: 0x2ffc59d32a524611bb891cab759112a51f9e33c0. FP mitigation clusters size now: 2
2022-12-30 16:53:49,584 - root - INFO - alert 0xe51eadf7db7eb56fe5dbb458ee63a269f7a807f7b508ad5df1aa742f71fd14c8 adding FP mitigation cluster: 0x2ffc59d32a524611bb891cab759112a51f9e33c0. FP mitigation clusters size now: 3
2022-12-30 16:53:51,098 - root - INFO - alert 0xbb95343df54204735c95bb6050c54d6bfa1ddb36e675b7a7a0846122acf740a2

In [24]:
#read output.txt and answer questions around
!grep 'anomaly' {name}.txt


#anomaly scores
#FP filtering


2022-12-30 15:53:20,919 - root - INFO - alert 0xb77076018fcc1e18bab4df662d180fba1b412ef40081102c4e9953040c6d7342 0xaf9ac4c204eabdd39e9b00f91c8383dc01ef1783e010763cad05cc39e82643bb LARGE-TRANSFER-OUT MoneyLaundering: 0xf3a465c9fa6663ff50794c698f600faa4b05c777 anomaly score of 0.7796312554872695
2022-12-30 15:53:51,395 - root - INFO - alert 0xf96919af4257d84fa4b7713b5bd495d2caeec9443c12cdeb5db77d91e8da3d1f 0x4adff9a0ed29396d51ef3b16297070347aab25575f04a4e2bd62ec43ca4508d2 POSSIBLE-MONEY-LAUNDERING-TORNADO-CASH MoneyLaundering: 0xf3a465c9fa6663ff50794c698f600faa4b05c777 anomaly score of 0.055347091932457786
2022-12-30 15:53:55,793 - root - INFO - alert 0x3a6afcddf21a284b8955c9db78a7a464e65378f758bf00397bad7759418a9daf 0xbc06a40c341aa1acc139c900fd1b7e3999d71b80c13a9dd50a369d8f923757f5 FLASHBOTS-TRANSACTIONS Exploitation: 0xf3a465c9fa6663ff50794c698f600faa4b05c777 anomaly score of 0.018632788747719783
2022-12-30 15:53:55,814 - root - INFO - Have sufficient number of alerts for 0xf3a465c9fa6